# IllinoisGRMHD: Basic equations and modules

## Authors: Zach Etienne & Leo Werneck

<font color='red'>**This module is currently under development**</font>

## This module introduces the basic equations solved by IllinoisGRMHD and provides an introduction to the modules inside the code.

## Introduction:

[IllinoisGRMHD](http://arxiv.org/abs/1501.07276) solves the equations of General Relativistic MagnetoHydroDynamics (GRMHD) using a high-resolution shock capturing scheme. It is a rewrite of the Illinois Numerical Relativity (ILNR) group's GRMHD code, and generates results that agree to roundoff error with that original code. Its feature set coincides with the features of the ILNR group's recent code (ca. 2009$-$2014), which was used in their modeling of the following systems:

1. Magnetized circumbinary disk accretion onto binary black holes
2. Magnetized black hole$-$neutron star mergers
3. Magnetized Bondi flow, Bondi-Hoyle-Littleton accretion
4. White dwarf$-$neutron star mergers

IllinoisGRMHD is particularly good at modeling GRMHD flows into black holes without the need for excision. Its [HARM-based conservative-to-primitive solver](https://arxiv.org/abs/astro-ph/0512420) has also been modified to check the physicality of conservative variables prior to primitive inversion, and move them into the physical range if they become unphysical.

Currently IllinoisGRMHD consists of

1. the Piecewise Parabolic Method (PPM) for reconstruction, 
2. the Harten, Lax, van Leer (HLL/HLLE) approximate Riemann solver, and
3. a modified HARM Conservative-to-Primitive solver. 

IllinoisGRMHD evolves the vector potential $A_{\mu}$ (on staggered grids) instead of the magnetic fields ($B^i$) irectly, to guarantee that the magnetic fields will remain divergenceless even at AMR boundaries. On uniform resolution rids, this vector potential formulation produces results equivalent to those generated using the standard, staggered flux-CT scheme. This scheme is based on that of [Del Zanna *et al.* (2003)](https://arxiv.org/abs/astro-ph/0210618).

### Required and recommended citations:
* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#basic_equations): **The equations of GRMHD**
1. [Step 2](#suitable_equations): **Recasting the equations of GRMHD into a more useful form**

<a id='basic_equations'></a>

# Step 1: The Equations of GRMHD \[Back to [top](#toc)\]
$$\label{basic_equations}$$

The basic equations solved by `IllinoisGRMHD` are

1. Einstein's equations: $R^{\mu\nu} + \frac{1}{2}g^{\mu\nu}R = 8\pi T^{\mu\nu} = 8\pi \left(T^{\mu\nu}_{\rm matter} + T^{\mu\nu}_{\rm EM}\right)$ ;
1. Conservation of baryon number: $\nabla_{\mu}\left(\rho_{0} u^{\mu}\right) = 0$ ;
1. Conservation of energy-momentum: $\nabla_{\mu}T^{\mu\nu} = \nabla_{\mu}\left(T^{\mu\nu}_{\rm matter} + T^{\mu\nu}_{\rm EM}\right) = 0$ ;
1. Homogeneous Maxwell's equations: $\nabla_{\nu} F^{*\mu\nu}=\frac{1}{\sqrt{-g}}\partial_{\nu}\left(\sqrt{-g}F^{*\mu\nu}\right)=0$ ,

where $g_{\mu\nu}$ is the ADM 4-metric, $R_{\mu\nu}$ and $R$ are the Ricci tensor and scalar, respectively, constructed from $g_{\mu\nu}$, $\nabla_{\mu}$ is the covariant derivative compatible with $g_{\mu\nu}$, $g\equiv\det\left(g_{\mu\nu}\right)$, $F^{\mu\nu}$ is the Faraday tensor and $F^{*\mu\nu}=\frac{1}{2}\epsilon^{\mu\nu\rho\sigma}F_{\mu\nu}$ its dual ($\epsilon^{\mu\nu\rho\sigma}$ is the Levi-Civita symbol).

The final equation that must be solved is the equation of state (EOS) for the matter. Currently `IllinoisGRMHD` implements a hybrid EOS of the form (**add citation**)

5. Equation of State (EOS): $P(\rho_{0},\epsilon) = P_{\rm cold}(\rho_{0}) + \left(\Gamma_{\rm th}-1\right) \rho_{0} \left[\epsilon-\epsilon_{\rm cold}(\rho_{0})\right]$



<a id='einstein_equations'></a>

## Step 1.1: Einstein's Equations
$$\label{einstein_equations}$$

`IllinoisGRMHD` solves Einstein's field equations (with $G_{\rm N}=1=c$) in the presence of matter sources,

$$
G^{\mu\nu} = 8\pi T^{\mu\nu}\ ,
$$

where $G^{\mu\nu} \equiv R^{\mu\nu} + \frac{1}{2}g^{\mu\nu}R$ is the Einstein tensor and the total energy-momentum tensor, $T^{\mu\nu}$, is the sum of the matter and electromagnetic energy-momentum tensors:

$$
T^{\mu\nu} = T^{\mu\nu}_{\rm matter} + T^{\mu\nu}_{\rm EM}\ .
$$

Einstein's field equations are solved using the BSSN formalism (see [this tutorial module](../../Tutorial-BSSN_formulation.ipynb) for a great overview).

<a id='conserv_of_baryon_number'></a>

## Step 1.2: Conservation of baryon number
$$\label{conserv_of_baryon_number}$$

We are also interested in the conservation of baryon number

$$
\nabla_{\mu}\left(\rho_{0} u^{\mu}\right) = 0\ ,
$$

where $\nabla_{\mu}$ is the covariant derivative associated with the 4-metric $g_{\mu\nu}$, $\rho_{0}$ is fluid rest-mass, and $u^{\mu}$ is the fluid 4-velocity.

<a id='conserv_of_em_tensor'></a>

## Step 1.3: Conservation of energy-momentum
$$\label{conserv_of_em_tensor}$$

$$
\nabla_{\mu}T^{\mu\nu} = \nabla_{\mu}\left(T^{\mu\nu}_{\rm matter} + T^{\mu\nu}_{\rm EM}\right) = 0\ .
$$

<a id='maxwell_equations'></a>

## Step 1.4: Homogeneous Maxwell’s equations
$$\label{maxwell_equations}$$

The electromagnetic field can be evolved in time using Maxwell's
